<a href="https://colab.research.google.com/github/capstone-dragon/LLM-Application/blob/master/LLM_Style_Guide_Editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup - Import Libraries

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.

#Load Falcon-7b Model
We will be fine tuning only a part of the mode, not the entirety. This will help on computation time and storage space.

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
   "Tell me a Story of a Dog named 'Max'",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq['generated_text']}")

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Tell me a Story of a Dog named 'Max'
Tell me a story of a dog named max,
I will listen to you all day long;
But, you have to be prepared with a story,
To tell me of a dog named max!
The story has to be very interesting.
For I love to hear the stories that are told;
But I have heard so many stories of dogs named 'Max'
I just want to hear your special story of a dog named max!
The story has to be true.
I don't like to be fooled by a lie;
If you are going to tell me a story of a dog named max,
It's got to be the truth; I can't take a lie!
Tell me a story of a dog named max,
Tell it to me, I want to hear.
Please tell me a story.
I'm ready to listen to your story


In [ ]:
sequences = pipeline(
   "What is the capital of the United States of America?",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: What is the capital of the United States of America?
Is it Washington? Is it New York?
Or maybe it’s a little city called Boston?
The answer may surprise you
The answer is none of the above. The capital of the United States of America is Washington, but its most famous city is actually New York.
The nation’s capital is Washington, D.C., but its most famous city is New York, New York.
The capital of the United States
Washington, D.C., is the capital of the United States. New York city is a city in the United States. Washington, DC is a city in the United States. New York City is a city in the United States. Washington, D.C. is the capital of the United States. Washington, D.C. is the capital of the United States. New York City, NY, is a city in New York. Washington, D.C. is a city


# Upload Dataset (csv file)

In [ ]:
# Upload file from Computer
from google.colab import files


uploaded = files.upload()

Saving Eng_to_STE - Sheet1.csv to Eng_to_STE - Sheet1.csv


In [ ]:
# Import Pandas Library
import pandas as pd
import io
# Read csv file
df = pd.read_csv(io.StringIO(uploaded['Eng_to_STE - Sheet1.csv'].decode('utf-8')))
df

,Input,Output
0,Follow the Safety Instructions,Obey the Safety Instructions
1,Grease the Fasteners,Apply grease to the fasteners
2,No leaks permitted,Make sure that there are no leaks
3,See if the pins are installed correctly,Make sure that the pins are properly installed.
4,Turn shaft assembly,Turn the shaft assembly
...,...,...
995,Unclip the visor.,Remove the visor from the clip.
996,Avoid uneven fuel load about the aircraft cent...,Make sure that the fuel load is equal on each ...
997,Unplug soldering iron.,Disconnect the soldering iron.
998,Do not use your hands to prevent movement of t...,Do not use your hands to prevent movement of t...
